In [2]:
import imageio
import numpy as np
from pathlib import Path
from omegaconf import OmegaConf
import os
from IPython.display import display, Image
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2 as cv
import matplotlib.pyplot as plt
import torch
import yaml
import shutil

def disp_im(im):
    plt.close()
    plt.cla()
    fig = plt.imshow(im)
    plt.close()
    display(fig.figure)


In [28]:
image_indices = dict(
    car=129,
    materials=144,
    hotdog=48,
    ficus=153,
    mic=189,
    ball=162,
    teapot=116,
    ship=140,
    toaster=162,
    coffee=157,
    helmet=156,
    lego=44,
    drums=188,
)


base_path = Path("../log/")
base_out_path = Path("../figures")
data_path = Path("/optane/nerf_datasets/nerf_synthetic")

exp = "noprednorms_nl0_conserve_pb0"
for p in (base_path / exp).glob("*"):
    print(p)
    config = OmegaConf.load(p / 'config.yaml')
    p = p / "imgs_test_all"
    dname = Path(config['dataset']['scenedir']).name
    if dname in image_indices:
        idx = image_indices[dname]
        out_path = base_out_path / dname
        out_path.mkdir(parents=True, exist_ok=True)
        pano_path = p / "envmaps"
        shutil.copyfile(pano_path / "mapped_pano.png", out_path / 'mapped_pano.png')
        shutil.copyfile(pano_path / "gt_pano.png", out_path / 'gt_pano.png')
        with (pano_path / 'coeffs.yaml').open('r') as f:
            coeffs = yaml.safe_load(f)

        shutil.copyfile(p / f"{idx:03d}.png", out_path / 'final.png')

        # copy GT
        gt_path = data_path / dname / "test"
        shutil.copyfile(gt_path / f"r_{idx}.png", out_path / 'gt_final.png')
        # get acc_map
        gt_acc_map = imageio.imread(gt_path / f"r_{idx}.png")[..., 3:4]

        gt_norm_path = gt_path / f"r_{idx}_normal.png"
        if gt_norm_path.exists():
            # mask out gray
            gt_norms = imageio.imread(gt_norm_path)
            gt_norms = (np.clip(gt_norms.astype(np.float32)/255 + 1-gt_acc_map.astype(np.float32)/255, 0, 1)*255).astype(np.uint8)
            imageio.imwrite(out_path / 'gt_normals.png', gt_norms)
            # shutil.copyfile(gt_norm_path, out_path / 'gt_normals.png')

        # spec, tint, diffuse all need mul with acc_map
        acc_map = imageio.imread(p / "acc_map" / f"{idx:03d}.png")
        acc_map = acc_map.reshape(*acc_map.shape[:2], 1)
        tint = imageio.imread(p / "tint" / f"{idx:03d}.exr")
        diffuse = imageio.imread(p / "diffuse" / f"{idx:03d}.png")
        spec = imageio.imread(p / "spec" / f"{idx:03d}.exr")
        spec = spec * np.array(coeffs['M']).reshape(1, 1, 3) + coeffs['B']
        spec = spec / (1 + spec)
        material = diffuse + 255*tint

        tint = np.concatenate(((255*tint + (255-acc_map)).astype(np.uint8), acc_map), axis=-1)
        imageio.imwrite(out_path / 'tint.png', tint)
        diffuse = np.concatenate((diffuse + (255-acc_map), acc_map), axis=-1)
        imageio.imwrite(out_path / 'diffuse.png', diffuse)
        spec = np.concatenate(((255*spec + (255-acc_map)).astype(np.uint8), acc_map), axis=-1)
        imageio.imwrite(out_path / 'spec.png', spec)
        material = np.concatenate((material + (255-acc_map), acc_map), axis=-1)
        imageio.imwrite(out_path / 'material.png', material.astype(np.uint8))

        normals = imageio.imread(p / "world_normal" / f"{idx:03d}.png")
        normals = np.concatenate((normals, acc_map), axis=-1)
        imageio.imwrite(out_path / 'normals.png', normals)

../log/noprednorms_nl0_conserve_pb0/mic_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_norussian_noprednorms_nl0


/tmp/ipykernel_2794597/1876719010.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  gt_acc_map = imageio.imread(gt_path / f"r_{idx}.png")[..., 3:4]
/tmp/ipykernel_2794597/1876719010.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  acc_map = imageio.imread(p / "acc_map" / f"{idx:03d}.png")
/tmp/ipykernel_2794597/1876719010.py:57: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  t

../log/noprednorms_nl0_conserve_pb0/helmet_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_noprednorms_nl0


/tmp/ipykernel_2794597/1876719010.py:49: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  gt_norms = imageio.imread(gt_norm_path)


../log/noprednorms_nl0_conserve_pb0/toaster_v38
../log/noprednorms_nl0_conserve_pb0/teapot_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_norussian_noprednorms_nl0
../log/noprednorms_nl0_conserve_pb0/ficus_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_norussian_noprednorms_nl0
../log/noprednorms_nl0_conserve_pb0/materials_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_norussian_noprednorms_nl0
../log/noprednorms_nl0_conserve_pb0/chair_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_norussian_noprednorms_nl0
../log/noprednorms_nl0_conserve_pb0/coffee_v38
../log/noprednorms_nl0_conserve_pb0/lego_v38
../log/noprednorms_nl0_conserve_pb0/hotdog_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_norussian_noprednorms_nl0
../log/noprednorms_nl0_conserve_pb0/ship_v38
../log/noprednorms_nl0_conserve_pb0/drums_v38
../log/noprednorms_nl0_conserve_pb0/shinyball_v37_mulLdotN_no2x_elam5e-5_max0.5rough_clip_pb0_noprednorms_nl0
../log/noprednorms_nl0_conserve_pb0/car_v38
../log/noprednorms_nl0_conser